# Time-variant global reliability sensitivity analysis of structures with both input random variables and stochastic processes : 
## Method validation on included numerical example.

In the paper above, their method is tested on a simple numerical example. 


In [1]:
import numpy as np
import openturns as ot
from numba import jit
from joblib import Parallel, delayed
from functools import partial

In [2]:
# the limit state function:
#@jit(nopython=True)
def g(t,x1, x2, y1):
    #function returns 1 if limit state is not exceeded on time interval
    out_g = np.sin(t)*x1 + np.square(x2) + 3*y1 - 5*(np.sin(t/8) + 1)
    return np.int(np.where(out_g<0,1,0).all())

# the function for multiprocessing on the limit state function
def g_mult(t,x1, x2, y1):
    # t is the time grid, and is invarient
    x1 = np.array(x1)
    N = x1.shape[0]
    x2 = np.array(x2)
    y1 = np.array(y1)
    output = Parallel(n_jobs=-1)(delayed(g) (t,x1[i], x2[i], y1[i]) for i in range(N))
    return ot.Point(output)


In [3]:
x = np.zeros((200,100))
uu = g_mult(x,x,x,x)

In [4]:
# Let's define a time grid. Let's say 100 samples over 10 seconds 
dimension = 1
NElem = [99]
mesher = ot.IntervalMesher(NElem)
lowerBound = [0] #s
upperBound = [10] #s
interval = ot.Interval(lowerBound,upperBound)
mesh = mesher.build(interval)
# 100 elements of 10 mm

In [5]:
# Process Y1:
model = ot.ExponentialModel()
algo = ot.KarhunenLoeveP1Algorithm(mesh, model)
algo.run()
kl_results = algo.getResult()
kl_results.setName('Y1')

# random Normals X1, X2
X1 = ot.Normal()
X1.setName('X1')
X2 = ot.Normal()
X2.setName('X2')

# The time grid :
t = np.array(mesh.getVertices())

In [6]:
# let's modify a bit our function as the time grid is invarient...
g_mod = partial(g_mult,t)

In [7]:
import KarhunenLoeveFieldSensitivity as klfs
list_kl_results = [X1, X2, kl_results]
AggregatedKLRes = klfs.AggregatedKarhunenLoeveResults(list_kl_results)

In [20]:
# initialization of the function wrapper : 
FUNC = klfs.KarhunenLoeveGeneralizedFunctionWrapper(
                                AggregatedKarhunenLoeveResults = AggregatedKLRes,
                                func        = None, 
                                func_sample = g_mod,
                                n_outputs   = 1)

In [33]:
xx=ot.SaltelliSensitivityAlgorithm()
xx.setDesign(sobolExp, response[0], 10000)

TypeError: InvalidArgumentException : Sobol experiment is too small (80000 vs 1040000)

In [28]:
response[0]

class=Sample name=Y_0 implementation=class=SampleImplementation name=Y_0 size=80000 dimension=1 data=[[0],[1],[0],...,[1],[1],[1]]

In [34]:
ot.RandomGenerator.SetSeed(111)
size = int(1e5)
experiment = klfs.KarhunenLoeveSobolIndicesExperiment(AggregatedKLRes, size, True)
sobolExp = experiment.generate()

Generating samples for the second order indices
Samples A and B of size 100000 and dimension 102
Experiment for second order generated
Experiment of size 800000 and dimension 102


In [35]:
response = FUNC(sobolExp)

Lifting as process sample
Using the batch evaluation function. Assumes that the outputs are in the
same order than for the single evaluation function. This one should only
return ProcessSamples, Samples, Lists or numpy arrays.
Element 0 of the output tuple returns samples of dimension 1


In [36]:
sensitivityAnalysis = klfs.SobolKarhunenLoeveFieldSensitivityAlgorithm(computeSecondOrder=True)
sensitivityAnalysis.setDesign(sobolExp, response, size)
sensitivityAnalysis.setEstimator(ot.SaltelliSensitivityAlgorithm())
sensitivityAnalysis.__nSobolIndices__

size initialized 800000
Means is
 [0.543026]
design_elem size, dim 800000 1
There are 3 indices to get for 1 outputs with dimensions 1 each.
Description all same? [X1_0,X2_0,Y1_0,Y1_1,Y1_2,Y1_3,Y1_4,Y1_5,Y1_6,Y1_7,Y1_8,Y1_9,Y1_10,Y1_11,Y1_12,Y1_13,Y1_14,Y1_15,Y1_16,Y1_17,Y1_18,Y1_19,Y1_20,Y1_21,Y1_22,Y1_23,Y1_24,Y1_25,Y1_26,Y1_27,Y1_28,Y1_29,Y1_30,Y1_31,Y1_32,Y1_33,Y1_34,Y1_35,Y1_36,Y1_37,Y1_38,Y1_39,Y1_40,Y1_41,Y1_42,Y1_43,Y1_44,Y1_45,Y1_46,Y1_47,Y1_48,Y1_49,Y1_50,Y1_51,Y1_52,Y1_53,Y1_54,Y1_55,Y1_56,Y1_57,Y1_58,Y1_59,Y1_60,Y1_61,Y1_62,Y1_63,Y1_64,Y1_65,Y1_66,Y1_67,Y1_68,Y1_69,Y1_70,Y1_71,Y1_72,Y1_73,Y1_74,Y1_75,Y1_76,Y1_77,Y1_78,Y1_79,Y1_80,Y1_81,Y1_82,Y1_83,Y1_84,Y1_85,Y1_86,Y1_87,Y1_88,Y1_89,Y1_90,Y1_91,Y1_92,Y1_93,Y1_94,Y1_95,Y1_96,Y1_97,Y1_98,Y1_99]#102
SobolIndicesName ['X1', 'X2', 'Y1']
Input Description is, ['X1', 'X2', 'Y1']


3

In [37]:
sensitivityAnalysis.getFirstOrderIndices()

Solving...
 size of samples:  800000
 number of indices to get 3


[[class=Point name=Sobol_Y_0_X1 dimension=1 values=[0.00945468],
  class=Point name=Sobol_Y_0_X2 dimension=1 values=[0.158129],
  class=Point name=Sobol_Y_0_Y1 dimension=1 values=[0.510099]]]

In [38]:
sensitivityAnalysis.getTotalOrderIndices()

[[class=Point name=TotalOrderSobol_Y_0_X1 dimension=1 values=[0.203894],
  class=Point name=TotalOrderSobol_Y_0_X2 dimension=1 values=[0.401444],
  class=Point name=TotalOrderSobol_Y_0_Y1 dimension=1 values=[0.83438]]]

In [19]:
display(sensitivityAnalysis.getSecondOrderIndices()[0])

This makes a problem: [class=SymmetricMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[0,-0.0115243,0.0574095,-0.0115243,0,0.124023,0.0574095,0.124023,0]]


class=SymmetricMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[0,-0.0115243,0.0574095,-0.0115243,0,0.124023,0.0574095,0.124023,0]

In [39]:
X  = sensitivityAnalysis.__results__[0]

In [41]:
Q = X.getFirstOrderIndicesDistribution()

In [46]:
Q.getMarginal(1).computeBilateralConfidenceInterval(0.95)

class=Interval name=Unnamed dimension=1 lower bound=class=Point name=Unnamed dimension=1 values=[0.151514] upper bound=class=Point name=Unnamed dimension=1 values=[0.164646] finite lower bound=[1] finite upper bound=[1]